In [ ]:
class BasicBlock:
    def __init__(self, leader, index, statements):
        self.leader = leader
        self.index = index
        self.statements = statements
        self.successors = []
        self.predecessors = []
        self.dominators = set()

    def __str__(self):
        successors_str = ', '.join(['B{}'.format(bb.index+1) for bb in self.successors])
        return 'B{}: contains: {}{}'.format(self.index+1, ', '.join(self.statements), successors_str)


def update_successors(basic_blocks):
    for i, bb in enumerate(basic_blocks):
        possible_successors = []
        if i < len(basic_blocks) - 1:
            possible_successors.append(basic_blocks[i+1])
        for j in range(i+1, len(basic_blocks)):
            if str(basic_blocks[j].leader) in bb.statements[-1]:
                possible_successors.append(basic_blocks[j])
        for successor in possible_successors:
            bb.successors.append(successor)
            successor.predecessors.append(bb)



def generate_blocks(three_address_code, leaders):
    basic_blocks = []
    leaders.append(len(three_address_code))
    leaders = sorted(leaders)
    for i in range(len(leaders)-1):
        block = three_address_code[leaders[i]:leaders[i+1]]
        basic_block = BasicBlock(leaders[i], i, block)
        basic_blocks.append(basic_block)

    # Update successors based on 'goto' statements
    update_successors(basic_blocks)

    return basic_blocks
    
def print_pfg(basic_blocks):
    for bb in basic_blocks:
        for successor in bb.successors:
            print('B{} -> B{}'.format(bb.index+1, successor.index+1))


# Sample input: Three Address Code statements
three_address_code = [
    "count = 0",
    "result = 0",
    "if count > 20 goto 7",
    "count = count + 1",
    "increment = 2 * count",
    "result = result + increment",
    "goto 3",
    "end"
]

# Finding leader statements
leaders = set()
leaders.add(0)  # The first statement is always a leader
for i in range(len(three_address_code)):
    if three_address_code[i].startswith('if'):
        # Instructions starting with 'if' and the next one of if
        leaders.add(i)
        target_block = int(three_address_code[i].split(' ')[-1]) # Extract target block from 'if' statement
        leaders.add(target_block)
        leaders.add(i + 1)
leaders = sorted(list(leaders))
for leader in leaders:
    print("{} : {}".format(leader, three_address_code[leader]))

# Generating basic blocks
basic_blocks = generate_blocks(three_address_code, leaders)

# Printing basic blocks
print("The basic blocks are:")
for basic_block in basic_blocks:
    print(basic_block)

print("\nThe program flow graph is:")
print_pfg(basic_blocks)

0 : count = 0
2 : if count > 20 goto 7
3 : count = count + 1
7 : end
The basic blocks are:
B1: contains: count = 0, result = 0B2
B2: contains: if count > 20 goto 7B3, B4
B3: contains: count = count + 1, increment = 2 * count, result = result + increment, goto 3B4
B4: contains: end

The program flow graph is:
B1 -> B2
B2 -> B3
B2 -> B4
B3 -> B4
